In [11]:
import customtkinter
import tkinter as tk

import logging

from pathlib import Path
from PIL import Image

logging.basicConfig(
    format="%(asctime)s %(levelname)s %(message)s",
    datefmt="%d/%m/%y %I:%M:%S %p",
    filename="logfile.log",
    encoding="utf-8",
    level=logging.DEBUG,
)


class MainApp(customtkinter.CTk):
    model_construters = dict(AlexNet=
    )

    def __init__(self):
        super().__init__()

        self.title("Demo")
        self.geometry("900*100")

        self.rowconfigure(0, weight=1)
        self.columnconfigure(1, weight=1)

        self.left_bar = customtkinter.CTkFrame(
            master=self, width=60, height=250, border_width=1
        )
        self.left_bar.grid(row=0, column=0, sticky="nesw")

        self.tabview = customtkinter.CTkTabview(
            master=self, width=300, height=250, border_width=1
        )
        self.tabview.grid(row=0, column=1, sticky="nesw")
        self.tabview.add("Data")
        self.tabview.add("Result")
        self.tabview.set("Data")

        # In left _bar
        self.left_bar.columnconfigure(0, weight=1)
        self.left_bar.rowconfigure(0, weight=1)

        self.closebutton = customtkinter.CTkButton(
            master=self.left_bar,
            command=self.close_window,
            text="close",
            fg_color="Red",
            anchor="center",
            border_width=1,
        )
        self.closebutton.grid(row=0, column=0, sticky="s", padx=10, pady=10)
        
        self.model_combobox = customtkinter.CTkComboBox(master=self.left_bar, values = MainApp.model_construters.keys(), command = self.model_combobox_callback)

        # In Right tab views tab(Data)
        self.tabview.tab("Data").rowconfigure((0, 1), weight=1)
        self.tabview.tab("Data").columnconfigure(0, weight=1)

        self.target_frame = FloderSelection(
            master=self.tabview.tab("Data"), text="Target", border_width=1
        )
        self.target_frame.grid(row=0, column=0, sticky="nesw", pady=5)

        self.query_frame = FloderSelection(
            master=self.tabview.tab("Data"), text="Query", border_width=1
        )
        self.query_frame.grid(row=1, column=0, sticky="nesw", pady=5)

        # In Right tab views tab(Result)
        # self.tabview.tab("Result").rowconfigure
        # self.tabview.tab("Result").columnconfigure
        self.result_window = ResultView(master=self.tabview.tab("Result"))

    def close_window(self) -> None:
        self.destroy()  # close the main apps window
        self.quit()  # quit python runtime
    
    def model_combobox_callback(self, choice):
        self.model = MainApp.model_construters[choice](weight = "DEFLAUT")

    def inference(self) -> None:
        self.tabview.set("Result")
        self.result_window.generate_results(self.model, self.target_frame.path, self.query_frame.path)

class FloderSelection(customtkinter.CTkFrame):
    def __init__(self, master, text, width=140, height=28, **kwarg):
        super().__init__(master, width, height, **kwarg)

        self.rowconfigure(1, weight=1)
        self.columnconfigure(1, weight=1)

        self.label = customtkinter.CTkLabel(master=self, text=text, anchor="center")
        self.label.grid(row=0, column=0, sticky="e", padx=5, pady=5)

        self.file_box = customtkinter.CTkEntry(master=self, height=28)
        self.file_box.grid(row=0, column=1, sticky="nesw")

        self.button = customtkinter.CTkButton(
            master=self, text="Select", width=84, command=self.ask_dict, anchor="center"
        )
        self.button.grid(row=0, column=2, padx=5, pady=5)

        # DLLM the tk.Text is using diff height unit, so set to 5
        self.preview_win = tk.Text(
            master=self, wrap="word", bg="black", height=5
        )  # use tk text wdiget for autop wrapping
        self.preview_win.grid(row=1, column=0, columnspan=3, sticky="nesw")

    @property
    def path(self) -> Path:
        "the path of selected floder"
        return Path(self.file_box.get())

    def ask_dict(self) -> None:
        dict_name = tk.filedialog.askdirectory(mustexist=True)
        self.file_box.insert("0", dict_name)
        logging.debug(f"Selected file {self.path}")
        self.create_preview()

    def create_preview(self):
        for i, file in enumerate(self.path.glob("*")):
            label = customtkinter.CTkLabel(
                master=self.preview_win,
                width=50,
                height=50,
                text="",
                image=customtkinter.CTkImage(
                    light_image=Image.open(file), size=(50, 50)
                ),
            )
            self.preview_win.window_create("end", window=label, padx=10, pady=10)

class ResultView(customtkinter.CTkFrame):
    def __init__(self,master, **kwarg):
        super().__init__(master, **kwarg)
    
    def generate_results(self, model, target_path, query_path):



app = MainApp()
logging.debug("Opened App")
app.mainloop()

In [4]:
from functools import wraps
def logged(func):
    # @wraps(func)
    def with_logging(*args, **kwargs):
        print(func.__name__ + " was called")
        return func(*args, **kwargs)
    return with_logging

@logged
def f(x):
   """does some math"""
   return x + x * x

print(f.__name__)
print(f.__doc__)   # prints 'does some math'

with_logging
None


In [1]:
from PIL import Image
import customtkinter
import os

IMAGE_WIDTH = 50
IMAGE_HEIGHT = 50
IMAGE_PATH = 'data/UNIQOExample/targets/31.jpg'

root = customtkinter.CTk()

your_image = customtkinter.CTkImage(light_image=Image.open(os.path.join(IMAGE_PATH)), size=(IMAGE_WIDTH , IMAGE_HEIGHT))
label = customtkinter.CTkLabel(master=root, image=your_image, text='')
label.grid(column=0, row=0)

root.mainloop()